In [1]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import typing
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1
seed=47

# Входные данные

In [2]:
def deviation_metric_one_sample(y_true: typing.Union[float, int], y_pred: typing.Union[float, int]) -> float:
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    if np.abs(deviation) <= THRESHOLD:
        return 0
    elif deviation <= - 4 * THRESHOLD:
        return 9 * NEGATIVE_WEIGHT
    elif deviation < -THRESHOLD:
        return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
    elif deviation < 4 * THRESHOLD:
        return ((deviation / THRESHOLD) - 1) ** 2
    else:
        return 9
    
def deviation_metric(y_true: np.array, y_pred: np.array) -> float:
    return np.array([deviation_metric_one_sample(y_true[n], y_pred[n]) for n in range(len(y_true))]).mean()


def deviation_metric_arr(y_true: np.array, y_pred: np.array) -> float:
    return np.array([deviation_metric_one_sample(y_true[n], y_pred[n]) for n in range(len(y_true))])

space_temp = np.linspace(0, 4, 200)
val_temp = 2
plt.plot([(val_temp - x) / val_temp for x in space_temp], [deviation_metric_one_sample(val_temp, x) for x in space_temp]);

In [3]:
train = pd.read_csv('/data/train.csv', low_memory=False)
test = pd.read_csv('/data/test.csv', low_memory=False)
sub = pd.read_csv('/data/test_submission.csv', low_memory=False)

print(train.shape)
train.head()

# Препроцессинг

## Дейттайм фичи

In [4]:
train['date'] = pd.to_datetime(train['date'], format="%Y-%m-%d")
test['date'] = pd.to_datetime(test['date'], format="%Y-%m-%d")

month_year = (train.date.dt.month + train.date.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.date.dt.month + test.date.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

week_year = (train.date.dt.weekofyear + train.date.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.date.dt.weekofyear + test.date.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

train['month'] = train.date.dt.month
train['dow'] = train.date.dt.dayofweek

test['month'] = test.date.dt.month
test['dow'] = test.date.dt.dayofweek

## Чистка этажей

In [6]:
train['floor'] = train['floor'].fillna('1')
test['floor'] = test['floor'].fillna('1')

train['street'] = train['street'].fillna('no street')
test['street'] = test['street'].fillna('no street')

y = train['per_square_meter_price']

In [7]:
def floor_recognition(s):
    if s[0] == "-":
        return -1 * floor_recognition(s[1:])
    if s.isdigit():
        return int(s)
    elif "подв" in s:
        return -1
    elif "цок" in s:
        return 0
    else:
        return -100
    
def floor_parser(row):
    #"<=0",  "2-5", "6-10", "11-20", "20<"
    result = [0, 0, 0, 0, 0]
    r = str(row).lower()
    for stop_word in [".0", "этажа", "этаж", "эт", " ", "-й"]:
        r = r.replace(stop_word, "")
    r = r.replace(".", ",").split(",")[0]
    floor = floor_recognition(r)
    if floor < -99:
        return pd.Series(result)
    elif floor < 1:
        result[0] = 1
    elif floor <=5:
        result[1] = 1
    elif floor <= 10:
        result[2] = 1
    elif floor <= 20:
        result[3] = 1
    else:
        result[4] = 1
    return pd.Series(result)

train[["floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20"]] = train["floor"].apply(floor_parser)
test[["floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20"]] = test["floor"].apply(floor_parser)

## Геофичи

In [8]:
import math

kremlin_lat, kremlin_lon = 55.753722, 37.620657

def dist_calc(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * \
          math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def check_offices(x):
    
    if x['osm_offices_points_in_0.001'] > 2 or x['osm_offices_points_in_0.005'] > 10 or x['osm_offices_points_in_0.0075'] > 15 or x['osm_offices_points_in_0.01'] > 20:
        return 1
    return 0

def check_financial_organizations(x):
    
    if x['osm_finance_points_in_0.001'] > 2 or x['osm_finance_points_in_0.005'] > 10 or x['osm_finance_points_in_0.0075'] > 15 or x['osm_finance_points_in_0.01'] > 20:
        return 1
    return 0


train['distance_to_kremlin'] = train.apply(
    lambda row: dist_calc(row.lat, row.lng, kremlin_lat, kremlin_lon), axis=1)
test['distance_to_kremlin'] = test.apply(
    lambda row: dist_calc(row.lat, row.lng, kremlin_lat, kremlin_lon), axis=1)

train['many_financial_organizations'] = train.apply(check_financial_organizations, axis=1)
test['many_financial_organizations'] = test.apply(check_financial_organizations, axis=1)
train['many_offices'] = train.apply(check_offices, axis=1)
test['many_offices'] = test.apply(check_offices, axis=1)

## Работа с городами

In [9]:
cities_1m_people = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск',
                    'Самара', 'Омск', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Воронеж', 'Пермь', 'Волгоград']

is_million = []
is_not_million = []

train['is_million_city'] = train['city'].apply(lambda x: 1 if x in cities_1m_people else 0)
test['is_million_city'] = test['city'].apply(lambda x: 1 if x in cities_1m_people else 0)

In [10]:
train['reform_house_population_500'] = train['reform_house_population_500'].fillna(train['reform_house_population_500'].median())
train['reform_house_population_1000'] = train['reform_house_population_1000'].fillna(train['reform_house_population_1000'].median())
train['reform_mean_floor_count_1000'] = train['reform_mean_floor_count_1000'].apply(lambda x: np.log(x+1))
train['reform_mean_floor_count_1000'] = train['reform_mean_floor_count_1000'].fillna(train['reform_mean_floor_count_1000'].mean())
train['reform_mean_floor_count_500'] = train['reform_mean_floor_count_500'].fillna(train['reform_mean_floor_count_500'].median())
train['reform_mean_year_building_1000'] = train['reform_mean_year_building_1000'].fillna(train['reform_mean_year_building_1000'].median())
train['reform_mean_year_building_500'] = train['reform_mean_year_building_500'].fillna(train['reform_mean_year_building_500'].median())
train['osm_city_nearest_population'] = train['osm_city_nearest_population'].fillna(train['osm_city_nearest_population'].median())

test['reform_house_population_500'] = test['reform_house_population_500'].fillna(train['reform_house_population_500'].median())
test['reform_house_population_1000'] = test['reform_house_population_1000'].fillna(train['reform_house_population_1000'].median())
test['reform_mean_floor_count_1000'] = test['reform_mean_floor_count_1000'].apply(lambda x: np.log(x+1))
test['reform_mean_floor_count_1000'] = test['reform_mean_floor_count_1000'].fillna(train['reform_mean_floor_count_1000'].mean())
test['reform_mean_floor_count_500'] = test['reform_mean_floor_count_500'].fillna(train['reform_mean_floor_count_500'].median())
test['reform_mean_year_building_1000'] = test['reform_mean_year_building_1000'].fillna(train['reform_mean_year_building_1000'].median())
test['reform_mean_year_building_500'] = test['reform_mean_year_building_500'].fillna(train['reform_mean_year_building_500'].median())
test['osm_city_nearest_population'] = test['osm_city_nearest_population'].fillna(train['osm_city_nearest_population'].median())

## Статистики относительно закодированых улиц

In [11]:
sq_by_street = train.groupby('street')['total_square'].apply(np.mean).to_dict()
train['square_by_street'] = train['street'].apply(lambda x: sq_by_street.get(x, np.nan))
test['square_by_street'] = test['street'].apply(lambda x: sq_by_street.get(x, np.nan))

mean_floor_500_by_street = train.groupby('street')['reform_mean_floor_count_500'].apply(np.mean).to_dict()
train['mean_floor_500_by_street'] = train['street'].apply(lambda x: mean_floor_500_by_street.get(x, np.nan))
test['mean_floor_500_by_street'] = test['street'].apply(lambda x: mean_floor_500_by_street.get(x, np.nan))

mean_year_500_by_street = train.groupby('street')['reform_mean_year_building_500'].apply(np.mean).to_dict()
train['mean_year_500_by_street'] = train['street'].apply(lambda x: mean_year_500_by_street.get(x, np.nan))
test['mean_year_500_by_street'] = test['street'].apply(lambda x: mean_year_500_by_street.get(x, np.nan))

## Выбор фичей в модель

In [12]:
drop_features = list({
#     "floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20",
#     'is_million_city', 'many_offices', 'many_financial_organizations',
    'square_by_street', 'mean_floor_500_by_street', 'mean_year_500_by_street', 
    'month_year_cnt', 'week_year_cnt', 'month', 'dow', 'distance_to_kremlin',
    'osm_train_stop_closest_dist'
})

used_cols = list({
    'total_square', 'osm_subway_closest_dist', 'distance_to_kremlin', 'lng',
    'lat', 'osm_city_closest_dist', 'reform_mean_floor_count_500',
    'reform_mean_floor_count_1000', 'osm_city_nearest_population',
    'square_by_street', 'reform_mean_year_building_500',
    'osm_transport_stop_closest_dist', 'osm_crossing_closest_dist',
    'floor<0', 'reform_house_population_500', 'osm_train_stop_closest_dist',
    'reform_mean_year_building_1000', 'osm_catering_points_in_0.005',
    'reform_house_population_1000', 'mean_floor_500_by_street',
    'osm_catering_points_in_0.01', 'reform_count_of_houses_1000',
    'osm_crossing_points_in_0.005', 'reform_count_of_houses_500',
    'osm_catering_points_in_0.0075', 'osm_hotels_points_in_0.0075',
    'osm_amenity_points_in_0.001', 'mean_year_500_by_street',
    'osm_healthcare_points_in_0.0075', 'week_year_cnt',
    'osm_catering_points_in_0.001', 'osm_hotels_points_in_0.005',
    'osm_finance_points_in_0.0075', 'osm_transport_stop_points_in_0.0075',
    'osm_crossing_points_in_0.01', 'osm_historic_points_in_0.01',
    'osm_shops_points_in_0.0075', 'osm_amenity_points_in_0.0075',
    'osm_crossing_points_in_0.0075', 'osm_crossing_points_in_0.001',
    'osm_culture_points_in_0.01', 'osm_finance_points_in_0.01',
    'osm_healthcare_points_in_0.01', 'osm_transport_stop_points_in_0.01',
    'osm_leisure_points_in_0.005', 'osm_hotels_points_in_0.01',
    'osm_culture_points_in_0.005', 'osm_amenity_points_in_0.01',
    'osm_offices_points_in_0.0075', 'osm_offices_points_in_0.005',
    'osm_culture_points_in_0.001', 'osm_shops_points_in_0.005',
    'osm_shops_points_in_0.001', 'osm_finance_points_in_0.001', 'floor<=5',
    'osm_amenity_points_in_0.005', 'osm_finance_points_in_0.005',
    'osm_shops_points_in_0.01', 'osm_train_stop_points_in_0.0075',
    'is_million_city', 'osm_culture_points_in_0.0075',
    'osm_historic_points_in_0.0075', 'osm_transport_stop_points_in_0.005',
    'osm_healthcare_points_in_0.005', 'osm_leisure_points_in_0.01',
    'osm_offices_points_in_0.01', 'osm_historic_points_in_0.005',
    'osm_building_points_in_0.01', 'osm_leisure_points_in_0.0075',
    'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
    'month_year_cnt', 'osm_offices_points_in_0.001',
    'osm_train_stop_points_in_0.005', 'month', 'floor<=10', 'floor<=20',
    'floor>20', 'many_financial_organizations', 'many_offices',
    'osm_building_points_in_0.001', 'osm_train_stop_points_in_0.01', 'dow'
})

all_cat_cols = ['city', 'floor', 'osm_city_nearest_name', 'region', 'street', 'realty_type', 'is_million_city', 'many_financial_organizations', 'many_offices', "floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20"]
cat_cols = ['region', 'city', 'realty_type', 'osm_city_nearest_name', 'street', 'is_million_city', 'many_financial_organizations', 'many_offices', "floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20"] 
cat_ohe = ['region', 'realty_type', 'osm_city_nearest_name', 'floor', 'osm_city_nearest_name']

In [15]:
train = train.drop(columns=train.columns[train.columns.str.startswith('OHE')])
test = test.drop(columns=test.columns[test.columns.str.startswith('OHE')])
for col in all_cat_cols:
    print(col)
    if col in cat_cols:
        oe_temp = OrdinalEncoder()
        oe_temp.fit(pd.concat([train[[col]], test[[col]]]))
        train[col + 'ord_enc'] = oe_temp.transform(train[[col]])
        test[col + 'ord_enc'] = oe_temp.transform(test[[col]])
    
    if col in cat_ohe:
        OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
        OH_encoder.fit(pd.concat([train[[col]], test[[col]]]))
        train = pd.concat([train, pd.DataFrame(OH_encoder.transform(train[[col]]), index=train.index).add_prefix('OHE_' + col + '_')], axis=1)
        test = pd.concat([test, pd.DataFrame(OH_encoder.transform(test[[col]]), index=test.index).add_prefix('OHE_' + col + '_')], axis=1)
    
    if col in cat_cols:
        train[col + '_cat'] = train[col].astype('category')
        test[col + '_cat'] = test[col].astype('category')

used_cols = list(set(used_cols) - set(drop_features))
cols_to_use = used_cols + train.columns[train.columns.str.startswith('OHE')].tolist()

# Моделинг

In [ ]:
def calc_kmeans(X, y, test, model, cv, used_cols, clusters=10, oof=False):
    res=[]
    local_probs=pd.DataFrame(index=test.index)
    two = ['lat','lng']
    for i, (tdx, vdx) in enumerate(cv.split(X, X['price_type'])):
        X_train, X_valid, y_train, y_valid = X.iloc[tdx].copy(), X.iloc[vdx].copy(), y[tdx].copy(), y[vdx].copy()
        
        kmeans = KMeans(n_clusters=clusters, random_state=seed)
        X_train['kmeans'] = kmeans.fit_predict(X_train[two])
        X_valid['kmeans'] = kmeans.predict(X_valid[two])
        
        if oof:
            test['kmeans'] = kmeans.predict(test[two])
        
        for km in X_train['kmeans'].unique():
            train_mask = X_train['kmeans']==km
            valid_mask = X_valid['kmeans']==km
            
            model.fit(X_train.loc[train_mask, used_cols], np.log(y_train[train_mask]))
            X_valid.loc[valid_mask, 'preds_km'] = np.exp(model.predict(X_valid.loc[valid_mask, used_cols]))
            
            if oof:
                test_mask = test['kmeans']==km
                local_probs.loc[test_mask, 'fold_%i'%i] = np.exp(model.predict(test.loc[test_mask, used_cols]))
            
        metric = deviation_metric(y_valid.tolist(), X_valid['preds_km'].tolist())
        res.append(metric)
        print(f'fold_{i+1}: {metric:.4f}')            
    
    mean_metric = round(np.mean(res), 4)
    print('mean:', mean_metric)
    if oof:
        local_probs['res'] = local_probs.mean(axis=1)
    return local_probs, mean_metric

## Наращиваем размер датасета с price_type==1. Ищем семплы с найменьшей ошибкой по предикту

In [20]:
lgmb1 = LGBMRegressor(
    random_seed = seed
)

manual = train[train['price_type']==1]
auto = train[train['price_type']==0]

model = lgmb1
model.fit(manual[cols_to_use], np.log(manual['per_square_meter_price']))
preds_aut = np.exp(model.predict(auto[cols_to_use]))
auto['metric_lgbm'] = deviation_metric_arr(auto['per_square_meter_price'].tolist(), preds_aut.tolist())

In [21]:
manual_full = pd.concat([
    manual, 
    auto[auto['metric_lgbm']==0], 
]).reset_index(drop=True)

X = manual_full.drop('per_square_meter_price', axis=1)
y = manual_full['per_square_meter_price']

sample_weight = np.r_[
    np.ones(manual.shape[0]), 
    np.ones(auto[auto['metric_lgbm']==0].shape[0]),
]
assert sample_weight.shape[0] == X.shape[0]

## Используем Лассо регрессию, чтобы отбросить незначимые фичи

In [24]:
from sklearn.linear_model import Lasso

cols_to_use = used_cols + train.columns[train.columns.str.startswith('OHE')].tolist() + [col + '_cat' for col in cat_cols]

np.set_printoptions(suppress=True)
lin_reg = Lasso()
lin_reg.fit(X[used_cols + train.columns[train.columns.str.startswith('OHE')].tolist()], y)
lin_reg.score(X[used_cols + train.columns[train.columns.str.startswith('OHE')].tolist()], y)
lasso_feat_drop = set(X[used_cols + train.columns[train.columns.str.startswith('OHE')].tolist()].columns[lin_reg.coef_ == 0])

In [25]:
lgmb = LGBMRegressor(
#     n_estimators=2000,
    objective='huber',
    random_seed = seed,
)

catb = CatBoostRegressor(
                iterations=700,
                random_state=seed,
                silent=True,
)

In [26]:
cols_to_use = list(set(cols_to_use) - lasso_feat_drop)

# kfold = StratifiedKFold(n_splits=5, shuffle = True, random_state = seed)
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=seed)

probs, mean_metric = calc_kmeans(
    X, y, test,
    catb, 
    kfold,
    cols_to_use,
    8,
    oof=True
)

# Важность фичей

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgmb.feature_importances_, cols_to_use)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:60])
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

# SUBMITION

In [26]:
sub['per_square_meter_price'] = probs['res']
sub.head()

In [27]:
plt.figure(figsize=(16, 8))
train['per_square_meter_price'].hist(bins=40, density=True, alpha=0.5)
sub['per_square_meter_price'].hist(bins=40, density=True, alpha=0.5)
(sub['per_square_meter_price'] * 0.9).hist(bins=40, density=True, alpha=0.5)

# Умножаем на константу, чтобы сделать распределения более похожими. Константа получена эвристическим методом

In [28]:
sub['per_square_meter_price'] = sub['per_square_meter_price'] * 0.9

In [29]:
sub.to_csv('submission0.1266_catb.csv', index=False)